In [162]:
import sklearn.linear_model
import pandas as pd
import matplotlib
import numpy as np
import pickle
from random import randint
from __future__ import division
import csv
import xgboost as xgb

In [3]:
fileObject = open('Data/pyVocab.pkl','r')
v = pickle.load(fileObject)  
v

['.',
 '!',
 'i',
 '"',
 'to',
 'the',
 ',',
 'a',
 'my',
 'and',
 'you',
 '?',
 'is',
 'it',
 'for',
 'in',
 'of',
 '0',
 'on',
 'me',
 'so',
 ';',
 '&',
 'have',
 'that',
 'but',
 "i'm",
 'just',
 'with',
 'be',
 'at',
 'not',
 'was',
 '-',
 'this',
 'now',
 'up',
 'good',
 'day',
 'get',
 'all',
 'out',
 'like',
 'are',
 'no',
 'go',
 'quot',
 '00',
 'today',
 'do',
 "it's",
 'too',
 'your',
 'love',
 'going',
 'work',
 'got',
 '\xc3',
 'lol',
 'time',
 ':',
 'am',
 'back',
 'from',
 '\xc2',
 'u',
 'what',
 'one',
 'will',
 'know',
 'we',
 'about',
 'im',
 'really',
 "don't",
 'amp',
 'had',
 'can',
 'see',
 'some',
 "can't",
 'its',
 "'",
 'if',
 'still',
 'night',
 'well',
 'new',
 'how',
 'want',
 '(',
 'think',
 'thanks',
 'home',
 'oh',
 'when',
 'as',
 'there',
 ')',
 '*',
 'more',
 'much',
 'here',
 'off',
 'they',
 'miss',
 'need',
 'last',
 'an',
 'then',
 '/',
 'hope',
 'been',
 'tomorrow',
 'great',
 'morning',
 'has',
 'or',
 'twitter',
 'haha',
 'her',
 'again',
 'feel'

In [5]:
len(v)

5000

In [45]:
X = np.load("Data/pyMatrix.npz")['X'][()] 
y = np.load("Data/pyMatrix.npz")['y'][()] 
print(X.shape)
print(y.shape)
print(len(y))

(1578627, 5000)
(1578627,)
1578627


In [46]:
y_test = y[0:50000]
y = y[50000:]

In [49]:
print(np.unique(y_test))
print(len(y_test))
print(np.unique(y))
print(len(y))
print(len(y) + len(y_test))

[-1]
50000
[0 1]
1528627
1578627


In [50]:
X_test = X[0:50000,]
X = X[50000:,]
print(X_test.shape)
print(X.shape)

(50000, 5000)
(1528627, 5000)


In [71]:
holdout_indices = np.random.choice(np.arange(1528627), size = 25477, replace = False)
holdout_indices

array([363334, 439707, 819940, ..., 990154, 960241, 123195])

In [83]:
temp = np.repeat(True, 1528627)
temp[holdout_indices] = False
train_indices = np.arange(1528627)[temp]
X_train = X[train_indices,]
X_holdout = X[holdout_indices,]
y_train = y[train_indices,]
y_holdout = y[holdout_indices,]

In [86]:
print(X_train.shape)
print(X_holdout.shape)
print(y_train.shape)
print(y_holdout.shape)
print(X_train.shape[0] + X_holdout.shape[0])

(1503150, 5000)
(25477, 5000)
(1503150,)
(25477,)
1528627


In [91]:
logit = sklearn.linear_model.LogisticRegression(penalty = 'l1')
logit.fit(X_train, y_train)
logit.fit

<bound method LogisticRegression.fit of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)>

In [96]:
yhat_logit = logit.predict(X_holdout)
yhat_logit

25477

In [100]:
(sum(yhat_logit == y_holdout))/len(yhat_logit)

0.79538407190799543

In [114]:
yhat = logit.predict(X_test)
id = np.arange(50000) + 1
result =  np.column_stack((id, yhat.astype(int)))
with open("logistic_submission.csv", "wb") as f:
    f.write(b'Id,y\n')
    np.savetxt(f, result, fmt='%i', delimiter=",")

In [136]:
id = np.arange(50000) + 1
1503150/25477

59.000274757624524

In [137]:
1503150 + 25477 + 50000

1578627

In [158]:
np.save("Data/X_train.npy", X_train)

In [159]:
example = np.load("Data/X_train.npy")[()]
example.shape

(1503150, 5000)

In [246]:
X_train = np.load("Data/X_train.npy")[()]
y_train = np.load("Data/y_train.npy")[()]
X_holdout = np.load("Data/X_holdout.npy")[()]
y_holdout = np.load("Data/y_holdout.npy")[()]

In [247]:
d_train = xgb.DMatrix(X_train, y_train)
d_holdout = xgb.DMatrix(X_holdout)

In [248]:
params = {"booster":"gblinear", "eta":"0.2", "objective":"binary:logistic"}

In [249]:
xg_model = xgb.train(params, d_train)

In [250]:
yhat_xg = xg_model.predict(d_holdout)

In [251]:
print(yhat_xg)
len(yhat_xg[yhat_xg < 0])

[ 0.57872611  0.52823019  0.9042204  ...,  0.77699602  0.80078173
  0.43662378]


0

In [252]:
yhat_xg[yhat_xg >= .5] = 1
yhat_xg[yhat_xg < .5] = 0

In [254]:
print(sum(yhat_xg == y_holdout)/len(yhat_xg))

0.792675746752


5